## 作業


In [1]:
import pandas as pd
google = pd.read_excel('https://raw.githubusercontent.com/ywchiu/tibame_tm/master/data/google_comments.xlsx',index_col=0)

In [2]:
google.head()

username  score  \
0                  倪依芙      5   
1                Mimi龍      4   
2                  林昌逸      1   
3                  林幸蓁      5   
4  台灣TAXI市區叫車禮車包車約拍找小鄭      5   

                                             comment  
0  起初找不到入口，還以為沒座位，但是原來入口在旁邊樓梯～座位區不大，但是料理很好吃～肉圓的醬微...  
1  聞名已久，今天特別來現場試試，目前改成點餐機點餐， ⋯⋯更多聞名已久，今天特別來現場試試，目...  
2          非常生氣服務員的態度😠服務很糟糕，麵都還沒吃完，一起身就把麵收走，擺明趕客人😡😡😡  
3  內用與外帶有不同的點餐方式內用須至店內使用機台點菜、輸入桌號並付款外帶好像只需在入口處點餐即...  
4  乾淨衛生，廁所很大在地下室，有免費飲料可以喝，拉麵愛好吃，麻辣豆腐不錯、肉粽不錯，目前吃過很...

In [12]:
def convertscore(score):
    if score >= 4:
        return 'good'
    elif score == 3:
        return 'soso'
    else:
        return 'bad'

In [13]:
google['status'] = google['score'].map(lambda e : convertscore(e))

In [14]:
import jieba
corpus = []
for e in google['comment'].tolist():
    corpus.append(list(jieba.cut(e)))

In [15]:
from gensim.models import word2vec
size = 50  # 產生多少維度 
min_count = 5 # 要算至少出現多少次數的字詞
workers = 4 # 使用多少個core 計算, -1 使用所有的core 進行計算 
window = 10  # 上下文的區間
iter = 1000 # 神經網路訓練的迭代數
sample = 1e-5 # 取樣的數量
model = word2vec.Word2Vec(corpus, 
                          workers = workers,
                          sample = sample,
                          size = size,
                          min_count=min_count,
                          window = window,
                          iter = iter)

In [21]:
model.wv.get_vector('肉圓')

array([-5.5977815e-01, -2.1862140e-01, -3.3223924e-01, -7.2246122e-01,
        1.2316134e+00,  9.8850006e-01, -3.5048631e-01, -2.2575563e-01,
        5.0253737e-01,  4.1952327e-01, -5.0696605e-01, -2.8036633e-01,
        5.5194068e-01,  5.0591475e-01, -1.8459958e-01,  4.0851516e-01,
       -3.0997938e-01, -1.7986359e-01,  4.2088670e-01,  8.0881453e-01,
       -5.7538080e-01,  8.8634163e-01, -6.2208039e-01,  1.4059737e-01,
       -1.8996248e-01, -3.7195459e-01,  2.9815900e-01, -1.1397655e+00,
       -7.2670674e-01, -3.6272144e-01,  1.6212463e-01,  4.0398489e-04,
       -4.9544394e-01, -4.4512048e-01,  5.5070603e-01, -6.4458750e-02,
        7.6481414e-01,  8.1949137e-02,  3.4438565e-01,  2.4917093e-01,
        2.3713836e-01, -1.3891813e-01, -2.9398248e-01, -4.8793319e-01,
        6.2633640e-01,  2.7992946e-01,  6.7572379e-01, -1.9740973e-01,
        1.0163072e-01, -2.7536762e-01], dtype=float32)

In [22]:
import jieba
corpus = []
tags = []
for idx,rec in google[google['status'].isin(['good', 'bad'])].iterrows():
    corpus.append(list(jieba.cut(rec.get('comment'))))
    tags.append(rec.get('status'))

In [23]:
import numpy as np
x = []
y = []
size = 50
for tag, s in zip(tags, corpus):
    vec = np.zeros(size).reshape((1, size))
    cnt = 0
    for w in s:
        if w in model:
            vec += model.wv.get_vector(w)
            cnt += 1
    if cnt > 0:
        x.append(vec / cnt)
        y.append(tag)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if __name__ == '__main__':


In [24]:
x = np.concatenate(x, axis = 0)

In [27]:
x.shape

(440, 50)

In [28]:
len(y)

440

In [32]:
train_x.shape

(352, 50)

In [33]:
len(train_y)

352

In [25]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [26]:
from sklearn.svm import SVC
clf = SVC(kernel = 'rbf')
clf.fit(train_x, train_y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [29]:
pred_y = clf.predict(test_x)

In [30]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, pred_y)

0.8636363636363636

In [31]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_y, pred_y)

array([[37,  3],
       [ 9, 39]])